Look at uncluster/Hernquist df

Need to sample tracer positions to follow power-law, then sample velocities from conditional distribution.

In [ ]:
import warnings

from astropy.constants import G as _G
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.optimize import root
import emcee
from gala.units import UnitSystem
from scipy.special import loggamma

In [ ]:
units = UnitSystem(u.mpc, u.yr, u.Msun, u.rad)
G = _G.decompose(units).value

In [ ]:
def density(r, gamma, a, b):
    return a**gamma * r**-gamma

def cdf(r, gamma, a, b):
    """CDF corresponding to the power-law density profile of tracers"""
    gg = 3 - gamma
    A = (b**gg - a**gg)**-1
    return A * (r**gg - a**gg)

In [ ]:
def sample_radii(cdf_func, size=1, a=0., b=np.inf, cdf_args=()):
    args = cdf_args + (a, b)
    def root_func(r, m):
        return m - cdf_func(float(r), *args)
    
    ms = np.random.uniform(0, 1, size=size)
    rs = []
    for m in ms:
        res = root(root_func, 1., args=(m,))
        rs.append(res.x[0])
        
    return np.array(rs)

In [ ]:
n_data = 512
Mbh = 4E6
gamma = 3.5
a = 2. # mpc
b = 1000. # mpc = 1 pc

In [ ]:
r = sample_radii(cdf, size=10000, a=a, b=b, cdf_args=(gamma,))

In [ ]:
bins = np.logspace(-3, 3, 32)
H,_ = np.histogram(r, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.

q = np.zeros((3,len(bin_cen)))
q[0] = bin_cen

fig,ax = plt.subplots(1, 1, figsize=(7,7))
ax.plot(bin_cen, density(bin_cen, gamma, a, b), 
         marker='', lw=2., ls='--')

ax.loglog(bin_cen, H/V/r.size, marker='')

ax.set_xlabel('$r$')
ax.set_ylabel('$n(r)$')

In [ ]:
from math import log

def potential(r, Mbh):
    return G*Mbh / r

def rv_to_E(r, v, Mbh):
    return -0.5*v**2 + potential(r, Mbh)

def v_c(r, Mbh):
    return np.sqrt(G * Mbh / r)

def log_df(E, L, Mbh, gamma, beta):
    g = gamma
    
    num = -2*beta*log(L) + (g - beta-1.5) * log(E) + beta*log(2)
    den = 1.5*log(2*np.pi) + (g - 2*beta) * log(G*Mbh)
    gams = loggamma(g - 2*beta + 1) - loggamma(g - beta - 0.5)

    return (num - den + gams).real

In [ ]:
def ln_vel_dist(p, r, Mbh, gamma):
    # HACK: IGNORE ANISOTROPY FOR NOW
    L = 1.
    beta = 0.
    
    v = p[0]
    if v <= 0.:
        return -np.inf
    
    E = rv_to_E(r, v, Mbh)
    if E < 0:
        return -np.inf
    
    return 2*log(v) + log_df(E, L, Mbh, gamma, beta)

In [ ]:
nwalkers = 32
v = np.zeros(n_data)

with warnings.catch_warnings():
    warnings.filterwarnings('error')
    
    for i in range(n_data):
        p0 = np.abs(np.random.normal(1E-1, 1E-3, (nwalkers,1)))
        sampler = emcee.EnsembleSampler(nwalkers=nwalkers, dim=1, 
                                        lnpostfn=ln_vel_dist, args=(r[i], Mbh, gamma))
        
        try:
            _ = sampler.run_mcmc(p0, 256)
        except Warning:
            print("Failed!", i)
            break
        v[i] = sampler.chain[0,-1,0]

In [ ]:
# for walker in sampler.chain[...,0]:
#     plt.plot(walker, marker='', drawstyle='steps', alpha=0.1)

In [ ]:
# S2 (star in GC) has this velocity-ish
((2*np.pi*980*u.au) / (15*u.yr)).to(u.mpc/u.yr)

In [ ]:
np.save('rv.npy', np.vstack((r[:n_data],v)))